In [1]:
%pip install -qU langchain-qdrant

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 8.2 MB/s eta 0:00:00


In [2]:
import os
import getpass # Import the getpass module
from google.colab import userdata


os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"

In [3]:
pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.5 MB/s eta 0:00:00


In [4]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [ ]:
!pip install -U langchain-community

"""
This code snippet installs necessary components and sets up tools
for converting text into numerical representations (embeddings)
using a powerful pre-trained model from Hugging Face.
These embeddings can then be used for various tasks, such as
semantic similarity search, text classification, and clustering.
"""
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

In [7]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

"""
this code block sets up a connection to a Qdrant vector
database, prepares to store documents there as vectors
using the Hugging Face embeddings model and creates an
empty list to fill with documents in further steps.
"""
url = userdata.get("QDRANT_URL")
api_key = userdata.get("QRANT_API_KEY")
docs = []  # put docs here
qdrant = QdrantVectorStore.from_documents(
    docs,
    embeddings,
    url=url,
    api_key=api_key,
    prefer_grpc=True,
    collection_name="my_documents",
)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

"""
Below are sample information on documents for our
understanding which we will insert into database
"""
document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled potatoes for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

qdrant.add_documents(documents=documents, ids=uuids)

In [14]:
"""
this code searches a Qdrant vector database
for the two documents most similar to the query
"LangChain provides abstractions to make working
with LLMs easy" and then prints the content and
metadata of these documents.
"""
results = qdrant.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy", k=2
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet', '_id': 'fb93846a-2e7f-4d72-bd91-3ce597747b2c', '_collection_name': 'my_documents'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet', '_id': 'fefb4b10-f6d6-4ce3-883a-1756d98a1dfd', '_collection_name': 'my_documents'}]


In [15]:
from langchain_qdrant import RetrievalMode
"""
this code block sets up a connection to a Qdrant vector
database, prepares to store documents there as vectors
using the Hugging Face embeddings model and creates an
empty list to fill with documents in further steps.
"""
qdrant = QdrantVectorStore.from_documents(
    docs,
    embedding=embeddings,
    url=url,
    api_key=api_key,
    prefer_grpc=True,
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.DENSE,
)

print("found_docs_with_score")
query = "Will it be hot tomorrow"
results = qdrant.similarity_search_with_score(query, k=1)

for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

found_docs_with_score
* [SIM=0.611811] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news', '_id': '57d2b4c9-c8e3-4166-bb7c-27205bade645', '_collection_name': 'my_documents'}]


In [16]:
from qdrant_client import models

"""
This code snippet is searching for documents
related to "Who are the best soccer players in
the world?" but it's adding an extra condition:
it only wants documents that contain the exact phrase
"The top 10 soccer players in the world right now."
in their content. Then, it prints the content and
source of the matching document. It's essentially performing
a targeted search within the vector database
"""
results = qdrant.similarity_search(
    query="Who are the best soccer players in the world?",
    k=1,
    filter=models.Filter(
        should=[
            models.FieldCondition(
                key="page_content",
                match=models.MatchValue(
                    value="The top 10 soccer players in the world right now."
                ),
            ),
        ]
    ),
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* The top 10 soccer players in the world right now. [{'source': 'website', '_id': '0e8b4493-60d6-41b6-b734-14330631c897', '_collection_name': 'my_documents'}]


In [17]:
"""
this code takes a query, uses a similarity search
to find the 2 most relevant documents in your database,
and then likely returns those documents.
"""
retriever = qdrant.as_retriever(search_type="similarity", search_kwargs={"k": 2})
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(metadata={'source': 'tweet', '_id': 'fb93846a-2e7f-4d72-bd91-3ce597747b2c', '_collection_name': 'my_documents'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'tweet', '_id': 'fefb4b10-f6d6-4ce3-883a-1756d98a1dfd', '_collection_name': 'my_documents'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]